Завдання до лабораторної роботи
Створити програму, яка зчитує заданий набір даних, виконує попередню
обробку та класифікацію документів відповідно до варіанту.

Варіант 1.
Файл bbc-news-data.csv. В якості текстової моделі використати TD-
IDF. Виконати класифікацію за допомогою алгоритмів наївний байєсів класифікатор та опорні вектори, порівняти їх точність.
Спробувати покращити модель наївний байєсів класифікатор за
допомогою GridSearchCV.

In [1]:
import numpy as np
import pandas as pd
import nltk
import re

from sklearn.datasets import fetch_20newsgroups
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [2]:
# імпортуємо таблицю з нестандартним роздільником
data = pd.read_csv("bbc-news-data.csv", header=0, sep='\t')
column_names = data.columns.tolist()

print(data.head(5))

   category filename                              title   
0  business  001.txt  Ad sales boost Time Warner profit  \
1  business  002.txt   Dollar gains on Greenspan speech   
2  business  003.txt  Yukos unit buyer faces loan claim   
3  business  004.txt  High fuel prices hit BA's profits   
4  business  005.txt  Pernod takeover talk lifts Domecq   

                                             content  
0   Quarterly profits at US media giant TimeWarne...  
1   The dollar has hit its highest level against ...  
2   The owners of embattled Russian oil giant Yuk...  
3   British Airways has blamed high fuel prices f...  
4   Shares in UK drinks and food firm Allied Dome...  


In [3]:
# за допомогою функції з поперідньої лабораторної обробляємо наш текст
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

def preproc_doc(doc):
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    tokens = wpt.tokenize(doc)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    doc = ' '.join(filtered_tokens)
    return doc
data['content'] = data['content'].apply(preproc_doc)
    
print(data.head(5))

   category filename                              title   
0  business  001.txt  Ad sales boost Time Warner profit  \
1  business  002.txt   Dollar gains on Greenspan speech   
2  business  003.txt  Yukos unit buyer faces loan claim   
3  business  004.txt  High fuel prices hit BA's profits   
4  business  005.txt  Pernod takeover talk lifts Domecq   

                                             content  
0  quarterly profits us media giant timewarner ju...  
1  dollar hit highest level euro almost three mon...  
2  owners embattled russian oil giant yukos ask b...  
3  british airways blamed high fuel prices drop p...  
4  shares uk drinks food firm allied domecq risen...  


In [4]:
# виконуємо пошук пустих документі, які могли виникнути під час обробки...
data = data.replace(r'^(\s?)+$', np.nan, regex=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  2225 non-null   object
 1   filename  2225 non-null   object
 2   title     2225 non-null   object
 3   content   2225 non-null   object
dtypes: object(4)
memory usage: 69.7+ KB


In [5]:
# видаляємо ці документи...
data = data.dropna().reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  2225 non-null   object
 1   filename  2225 non-null   object
 2   title     2225 non-null   object
 3   content   2225 non-null   object
dtypes: object(4)
memory usage: 69.7+ KB


In [6]:
# розділяємо дані на тренувальну та тестову вибірки
train_category, test_category, train_content, test_content = train_test_split(
    np.array(data['category']),
    np.array(data['content']),
    test_size=0.2,
    random_state=42
)

# ініціалізуємо TF-IDF трансформатор
tfidf = TfidfVectorizer()

# обчислюємо TF-IDF для тренувальної вибірки та трансформуємо дані
train_tfidf = tfidf.fit_transform(train_content)

# трансформуємо тестову вибірку з використанням попередньо натренованої моделі
test_tfidf = tfidf.transform(test_content)

# перевіряємо розмір тренувальної та тестової вибірок
train_tfidf.shape, test_tfidf.shape


((1780, 28280), (445, 28280))

In [7]:
# натренуємо наївний байєсів класифікатор та здійснимо класифікацію тестових даних
nb_classifier = MultinomialNB()
nb_classifier.fit(train_tfidf, train_category)
nb_predicted = nb_classifier.predict(test_tfidf)

# оцінимо точність наївного байєсівого класифікатора
nb_accuracy = accuracy_score(test_category, nb_predicted)
print("Точність наївного байєсівого класифікатора:", nb_accuracy)

# натренуємо опорні вектори та здійснимо класифікацію тестових даних
svm_classifier = SVC(kernel='linear', random_state=42)
svm_classifier.fit(train_tfidf, train_category)
svm_predicted = svm_classifier.predict(test_tfidf)

# оцінимо точність опорних векторів
svm_accuracy = accuracy_score(test_category, svm_predicted)
print("Точність опорних векторів:", svm_accuracy)

Точність наївного байєсівого класифікатора: 0.9617977528089887
Точність опорних векторів: 0.9730337078651685


In [8]:
# створюємо класифікатор наївного байєсівського класифікатора та параметри для GridSearchCV
nb_classifier = MultinomialNB()
nb_params = {'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
             'clf__alpha': [0.01, 0.1, 1.0]}

# створюємо пайплайн для класифікатора наївного байєсівського класифікатора та TF-IDF векторизатора
nb_pipeline = Pipeline([('vect', TfidfVectorizer()), ('clf', nb_classifier)])

# застосовуємо GridSearchCV для знаходження найкращих параметрів для класифікатора наївного байєсівського класифікатора
nb_grid_search = GridSearchCV(nb_pipeline, nb_params, cv=10, n_jobs=-1)
nb_grid_search.fit(train_content, train_category)

# передбачаємо класифікацію на тестовій вибірці
nb_predicted = nb_grid_search.predict(test_content)

# обчислюємо точність класифікації
nb_accuracy = accuracy_score(test_category, nb_predicted)
print("Точність оновленого баєсівого класифікатору:", nb_accuracy)

Точність оновленого баєсівого класифікатору: 0.9752808988764045
